In [53]:
import pandas as pd
import numpy as np

In [54]:
Lab = pd.read_csv("LabReduced.csv", usecols = ['Patient_ID', 'PerformedDate', 'Name_calc', 'TestResult_calc', 'UnitOfMeasure_calc'])
Lab.PerformedDate = pd.to_datetime(Lab.PerformedDate,format='%Y-%m-%d')
Exam = pd.read_csv("Exam.csv", usecols = ['Patient_ID', 'Exam1', 'Result1_calc', 'Exam2', 'Result2_calc', 'UnitOfMeasure_calc', 'DateCreated'])
Exam.DateCreated = pd.to_datetime(Exam.DateCreated,format='%Y-%m-%d')
DiseaseCase = pd.read_csv("DiseaseCase.csv")

C:\Users\niche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [55]:
# DiseaseCase
DiseaseCase['Disease'] = DiseaseCase['Disease'].replace("Diabetes Mellitus (ml)","Diabetes Mellitus") # Make all DM2 diagnosis' the same
DiseaseCase['DateOfOnset'] = pd.to_datetime(DiseaseCase.DateOfOnset,format='%Y-%m-%d')
DiseaseCase = DiseaseCase.sort_values(by='DateOfOnset', ascending = True)
DiseaseCase = DiseaseCase.drop_duplicates(subset = ['Patient_ID'])

In [56]:
Exam1 = Exam.drop(columns=['Exam2','Result2_calc']).rename(columns={'Exam1':'Name_calc','Result1_calc':'TestResult_calc'})
Exam2 = Exam.drop(columns=['Exam1','Result1_calc']).rename(columns={'Exam2':'Name_calc','Result2_calc':'TestResult_calc'})
Exam = pd.concat([Exam1,Exam2]).dropna()
print(Exam['Name_calc'].value_counts(ascending=False))

dBP (mmHg)                  2570192
sBP (mmHg)                  2570192
Weight (kg)                 1102589
BMI (kg/m^2)                 910459
Height (cm)                  736942
Waist Circumference (cm)     115353
PEFR (L/min)                     35
Name: Name_calc, dtype: int64


In [57]:
tests = ['Alanine Aminotransferase in Serum or Plasma (ALT)',
              'Aspartate Aminotransferase in Serum or Plasma (AST)',
              'Urea (mM)',
              'Plasma Creatinine Clearance (eGFR)',
              'Potassium (mM)',
              'Sodium (mM)',
              'Gamma Glutamyl Transferase in Serum or Plasma (GGT)',
              'Leukocytes in Blood (WBC)',
              'Protein, total (g/L)',
              'Total Cholesterol (TCh)',
              'Total Bilirubin (TBIL)',
              'Platelets in Blood (PLT)',
              'Fasting Blood Glucose (FBG)',
              'Hemoglobin (Hb)',
              'Erythrocytes in Blood (RBC)',
              'Albumin (g/L)',
              'Hemoglobin A1c (HbA1c)',
              'BMI (kg/m^2)']
Lab = Lab.loc[Lab.Name_calc.isin(tests)]
Exam = Exam.loc[Exam.Name_calc.isin(tests)]

In [58]:
Exam = Exam.rename(columns={'DateCreated':'PerformedDate'})
Exam = Exam[Exam.columns[[0,4,1,2,3]]]
Exam = Exam.drop_duplicates(subset=['Patient_ID','PerformedDate'])

In [59]:
lab = pd.concat([Lab,Exam])

In [60]:
lab.isna().sum()

Patient_ID                 0
PerformedDate         432244
Name_calc                  0
TestResult_calc            0
UnitOfMeasure_calc     28971
dtype: int64

In [61]:
lab = lab.dropna(subset = ['PerformedDate'])
lab.isna().sum()

Patient_ID                0
PerformedDate             0
Name_calc                 0
TestResult_calc           0
UnitOfMeasure_calc    28660
dtype: int64

In [62]:
disease_periods = pd.read_csv('AP_posnegIDS.csv', index_col=None)

disease_periods.DateOfOnset = pd.to_datetime(disease_periods.DateOfOnset,format='%Y-%m-%d')
disease_periods.CompOnset = pd.to_datetime(disease_periods.CompOnset,format='%Y-%m-%d')

disease_periods = disease_periods.drop(disease_periods.columns[0], axis=1)
neg_sample = disease_periods.iloc[1902:3804]
disease_periods = disease_periods.iloc[0:1902]

In [63]:
disease_periods.head()

,Patient_ID,DateOfOnset,CompOnset
0,1000100165845,2006-02-14,2013-03-04
1,1000100082336,2004-10-18,2008-04-08
2,1000100035828,2004-03-09,2009-09-28
3,1000100033944,2005-06-17,2007-03-16
4,1000100166298,2013-01-30,2017-01-18


In [64]:
patient_records = lab.loc[lab['Patient_ID'].isin(disease_periods['Patient_ID'])]
patient_records = pd.merge(patient_records, disease_periods, on='Patient_ID')
print(len(patient_records.index),patient_records.columns)

243260 Index(['Patient_ID', 'PerformedDate', 'Name_calc', 'TestResult_calc',
       'UnitOfMeasure_calc', 'DateOfOnset', 'CompOnset'],
      dtype='object')


In [65]:
patient_records.head()

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset,CompOnset
0,1000100166298,2016-04-23,Leukocytes in Blood (WBC),8.7,10^9/L,2013-01-30,2017-01-18
1,1000100166298,2017-10-19,Total Cholesterol (TCh),2.25,mmol/L,2013-01-30,2017-01-18
2,1000100166298,2019-07-26,Erythrocytes in Blood (RBC),5.5,10^12/L,2013-01-30,2017-01-18
3,1000100166298,2018-12-12,Platelets in Blood (PLT),352,10^9/L,2013-01-30,2017-01-18
4,1000100166298,2018-08-15,Platelets in Blood (PLT),302,10^9/L,2013-01-30,2017-01-18


In [96]:
relVisits = patient_records[(patient_records['PerformedDate'] < patient_records['CompOnset']) & (patient_records['PerformedDate'] >= patient_records['DateOfOnset'])]
len(relVisits.index)

77217

In [97]:
# Count number of visits per patient
df = relVisits[['Patient_ID','PerformedDate']].groupby('Patient_ID').nunique() < 51
good = df.index[df['PerformedDate'] == True].tolist()
n = len(good)
print(n)
relVisits = relVisits[relVisits['Patient_ID'].isin(good)]

1688


In [98]:
relVisits.PerformedDate = pd.to_datetime(relVisits.PerformedDate,format='%Y-%m-%d')
relVisits.CompOnset = pd.to_datetime(relVisits.CompOnset,format='%Y-%m-%d')
relVisits.DateOfOnset = pd.to_datetime(relVisits.DateOfOnset,format='%Y-%m-%d')

In [99]:
print(relVisits.isna().sum())
print(relVisits.dtypes)

Patient_ID            0
PerformedDate         0
Name_calc             0
TestResult_calc       0
UnitOfMeasure_calc    0
DateOfOnset           0
CompOnset             0
dtype: int64
Patient_ID                     int64
PerformedDate         datetime64[ns]
Name_calc                     object
TestResult_calc               object
UnitOfMeasure_calc            object
DateOfOnset           datetime64[ns]
CompOnset             datetime64[ns]
dtype: object


In [100]:
# Remove patients with < 5 results
#print(Results.groupby('Patient_ID').agg('count'))
n= relVisits['Patient_ID'].unique()
print(len(n))
print(len(relVisits.index) / len(relVisits['Patient_ID'].unique())) # Average visits per person
#len(a.index)

1688
40.37677725118483


In [101]:
relVisits['dp'] = relVisits['CompOnset'] - relVisits['DateOfOnset']
dp = relVisits['dp'].mean()
print(dp)

1961 days 05:34:07.091965504


In [102]:
pos_patients = relVisits.Patient_ID.unique()
len(pos_patients)

1688

In [103]:
# Negative sample part 1
from random import sample

PatientInfo = pd.read_csv('Patient.csv')

pos_patients = relVisits.Patient_ID.unique()
all_neg_patients = lab.loc[~lab.Patient_ID.isin(pos_patients)].Patient_ID.unique()

neg_patients = pd.Series([], dtype = 'int64')
size = len(pos_patients)
#takes 45 sec ish to run
for i in range(size):
    refPatient_ID = pos_patients[i]
    refYOB = PatientInfo.loc[PatientInfo.Patient_ID == refPatient_ID].BirthYear.iloc[0]
    refSex = PatientInfo.loc[PatientInfo.Patient_ID == refPatient_ID].Sex.iloc[0]
    PossiblePatients = PatientInfo.loc[PatientInfo.Patient_ID.isin(all_neg_patients) 
                                       & (PatientInfo.Sex == refSex)]
    selectedPat = False
    yearGap = 0
    while not selectedPat:
        #print(len(PossiblePatients))
        possible = PossiblePatients.loc[(PossiblePatients.BirthYear >= refYOB - yearGap) 
                                        & (PossiblePatients.BirthYear <= refYOB + yearGap)].Patient_ID
        if len(possible > 0):
            selectedPat = possible.sample().iloc[0]
            neg_patients = neg_patients.append(pd.Series([selectedPat], dtype = 'int64'), ignore_index = True)
            all_neg_patients = all_neg_patients[all_neg_patients != selectedPat]
        yearGap += 1

neg_patients.head()

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680


0     4000000266103
1     1000100023650
2    10001000141654
3     1000100018552
4     1000100572965
dtype: int64

Int64Index([ 1000100000014,  1000100000037,  1000100000078,  1000100000156,
             1000100000224,  1000100000249,  1000100000266,  1000100000287,
             1000100000295,  1000100000369,
            ...
            12120120077673, 12120120077700, 12120120077706, 12120120077845,
            12120120077960, 12120120078059, 12120120078086, 12120120078151,
            12120120078318, 12120120078418],
           dtype='int64', name='Patient_ID', length=142692)

In [115]:
#Negative sample part 2
rest = lab[['Patient_ID','PerformedDate']].groupby('Patient_ID').nunique() < 51
negGood = rest.loc[rest.PerformedDate].index.to_series()
neg_samp = negGood.loc[negGood.isin(neg_patients)]
print(len(neg_samp))
neg_rec = lab.loc[lab['Patient_ID'].isin(neg_samp)]
neg_rec = pd.merge(neg_rec, DiseaseCase[['Patient_ID','DateOfOnset']], on='Patient_ID')
print(len(neg_rec.index)/ len(neg_samp))
neg_rec = neg_rec[neg_rec['PerformedDate'] > neg_rec['DateOfOnset']]
print(len(neg_rec.index)/ len(neg_samp))
neg_rec = neg_rec[(neg_rec['PerformedDate'] > neg_rec['DateOfOnset']) & (neg_rec['PerformedDate'] < neg_rec['DateOfOnset']+dp) ]
print(len(neg_rec.index)/ len(neg_samp))

1510
74.5158940397351
57.64503311258278
40.19072847682119


In [116]:
len(neg_samp)

1510

In [117]:
print(len(relVisits.index))
print(len(neg_rec.index))
relVisits = relVisits.append(neg_rec, ignore_index=True)
print(len(relVisits.index))
print(relVisits.isna().sum())
print(relVisits.dtypes)

68156
60688
128844
Patient_ID                0
PerformedDate             0
Name_calc                 0
TestResult_calc           0
UnitOfMeasure_calc       49
DateOfOnset               0
CompOnset             60688
dp                    60688
dtype: int64
Patient_ID                      int64
PerformedDate          datetime64[ns]
Name_calc                      object
TestResult_calc                object
UnitOfMeasure_calc             object
DateOfOnset            datetime64[ns]
CompOnset              datetime64[ns]
dp                    timedelta64[ns]
dtype: object


In [118]:
combined = relVisits.drop(['CompOnset','dp'],axis='columns')
print(combined.isna().sum())
print(combined.dtypes)
combined.TestResult_calc = pd.to_numeric(combined.TestResult_calc,errors='coerce')
combined = combined.dropna()
print(len(combined.index))

Patient_ID             0
PerformedDate          0
Name_calc              0
TestResult_calc        0
UnitOfMeasure_calc    49
DateOfOnset            0
dtype: int64
Patient_ID                     int64
PerformedDate         datetime64[ns]
Name_calc                     object
TestResult_calc               object
UnitOfMeasure_calc            object
DateOfOnset           datetime64[ns]
dtype: object
127441


In [119]:
#remove vast minority units (mg/L for Total Bilirubin (n = 40) and mmol/L for Hemoglobin (n = 10))
badIndices = combined.loc[(combined.Name_calc == 'Total Bilirubin (TBIL)') & (relVisits.UnitOfMeasure_calc == 'mg/L')].index
badIndices2 = combined.loc[(combined.Name_calc == 'Hemoglobin (Hb)') & (relVisits.UnitOfMeasure_calc == 'mmol/L')].index
combined = combined.drop(badIndices)
combined = combined.drop(badIndices2)

In [120]:
Results = combined
print(len(Results.index))

127441


In [121]:
for test in tests:
    q1 = Results.loc[Results.Name_calc == test].TestResult_calc.quantile(.25)
    q3 = Results.loc[Results.Name_calc == test].TestResult_calc.quantile(.75)
    rng = q3 - q1
    badIndices = Results.loc[(Results.Name_calc == test) & ((Results.TestResult_calc < (q1 - rng)) 
                                    |(Results.TestResult_calc > (q3 + rng)))].index
    Results = Results.drop(badIndices)

In [122]:
print(len(Results.index))
Results.head()

118771


,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset
0,1000100166298,2016-04-23,Leukocytes in Blood (WBC),8.7,10^9/L,2013-01-30
1,1000100166298,2014-04-08,Hemoglobin A1c (HbA1c),9.2,%,2013-01-30
2,1000100166298,2015-09-29,Hemoglobin A1c (HbA1c),7.5,%,2013-01-30
3,1000100166298,2014-10-31,Hemoglobin A1c (HbA1c),7.9,%,2013-01-30
4,1000100166298,2014-07-23,Hemoglobin A1c (HbA1c),8.8,%,2013-01-30


In [123]:
turtiles = pd.read_csv('TestTurtiles.csv')

In [124]:
turtiles

,Unnamed: 0,TestName,t1,t2
0,0,Alanine Aminotransferase in Serum or Plasma (ALT),18.000000,27.00
1,1,Aspartate Aminotransferase in Serum or Plasma ...,18.000000,24.00
2,2,Urea (mM),5.100000,6.70
3,3,Plasma Creatinine Clearance (eGFR),60.000000,83.00
4,4,Potassium (mM),4.200000,4.60
5,5,Sodium (mM),139.000000,141.00
6,6,Gamma Glutamyl Transferase in Serum or Plasma ...,23.000000,41.00
7,7,Leukocytes in Blood (WBC),6.200000,7.90
8,8,"Protein, total (g/L)",63.000000,71.00
9,9,Total Cholesterol (TCh),3.530000,4.39


In [125]:
for index, row in Results.iterrows() :
    test = row['Name_calc']
    value = row['TestResult_calc']
    #feature = [0,0,0]
    t1 = float(turtiles.loc[turtiles['TestName'] == test]['t1'])
    t2 = float(turtiles.loc[turtiles['TestName'] == test]['t2'])
    #print(t1,t2,value)
    if value < t1 :
        feature = 1.0 #[1,0,0]
    elif value > t2 :
        feature = 3.0 #[0,0,1]
    else :
        feature = 2.0 #[0,1,0]
    Results.at[index, 'TestResult_calc'] = feature
    
        

In [126]:
Results.head()

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset
0,1000100166298,2016-04-23,Leukocytes in Blood (WBC),3.0,10^9/L,2013-01-30
1,1000100166298,2014-04-08,Hemoglobin A1c (HbA1c),3.0,%,2013-01-30
2,1000100166298,2015-09-29,Hemoglobin A1c (HbA1c),3.0,%,2013-01-30
3,1000100166298,2014-10-31,Hemoglobin A1c (HbA1c),3.0,%,2013-01-30
4,1000100166298,2014-07-23,Hemoglobin A1c (HbA1c),3.0,%,2013-01-30


In [127]:
Results.groupby('TestResult_calc').agg('count')

,Patient_ID,PerformedDate,Name_calc,UnitOfMeasure_calc,DateOfOnset
TestResult_calc,,,,,
1.0,38315,38315,38315,38315,38315
2.0,43352,43352,43352,43352,43352
3.0,37104,37104,37104,37104,37104


In [128]:
"""Results.PerformedDate = pd.to_datetime(Results.PerformedDate,format='%Y-%m-%d')
Results = Results.sort_values(by='PerformedDate', ascending = True)
Results.head()"""

"Results.PerformedDate = pd.to_datetime(Results.PerformedDate,format='%Y-%m-%d')\nResults = Results.sort_values(by='PerformedDate', ascending = True)\nResults.head()"

In [129]:
Results = Results.groupby('Patient_ID').apply(lambda g: g.sort_values(by = 'PerformedDate', ascending =True))
Results = Results.reset_index(drop = True)
Results.head()

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset
0,1000100000266,2006-02-07,Plasma Creatinine Clearance (eGFR),1.0,mL/min,2006-01-31
1,1000100000266,2006-02-07,Total Cholesterol (TCh),1.0,mmol/L,2006-01-31
2,1000100000266,2006-02-07,Fasting Blood Glucose (FBG),1.0,mmol/L,2006-01-31
3,1000100000266,2006-02-07,Alanine Aminotransferase in Serum or Plasma (ALT),3.0,U/L,2006-01-31
4,1000100000266,2006-02-07,Gamma Glutamyl Transferase in Serum or Plasma ...,2.0,U/L,2006-01-31


In [130]:
rem = []
for index, row in Results.iterrows() :
    temp = row[['Patient_ID', 'Name_calc','TestResult_calc']]
    if index == 0 :
        prev = temp
        continue
    if temp.equals(prev):
        rem.append(index)
    prev = temp
    #print(prev)

print(len(rem))


5493


In [131]:
Results_u = Results
print(len(Results_u.index))
Results_u = Results_u.drop(rem)
print(len(Results_u.index))

118771
113278


In [132]:
Results_u.head(40)

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset
0,1000100000266,2006-02-07,Plasma Creatinine Clearance (eGFR),1.0,mL/min,2006-01-31
1,1000100000266,2006-02-07,Total Cholesterol (TCh),1.0,mmol/L,2006-01-31
2,1000100000266,2006-02-07,Fasting Blood Glucose (FBG),1.0,mmol/L,2006-01-31
3,1000100000266,2006-02-07,Alanine Aminotransferase in Serum or Plasma (ALT),3.0,U/L,2006-01-31
4,1000100000266,2006-02-07,Gamma Glutamyl Transferase in Serum or Plasma ...,2.0,U/L,2006-01-31
5,1000100000266,2006-02-07,Leukocytes in Blood (WBC),2.0,10^9/L,2006-01-31
6,1000100000266,2006-02-07,Erythrocytes in Blood (RBC),3.0,10^12/L,2006-01-31
7,1000100000266,2006-02-07,Hemoglobin (Hb),3.0,g/L,2006-01-31
8,1000100000266,2006-02-07,Platelets in Blood (PLT),1.0,10^9/L,2006-01-31
9,1000100000266,2006-02-07,Hemoglobin A1c (HbA1c),1.0,%,2006-01-31


In [133]:
AllVisits = {}
VisitValues = {}
for index, row in Results_u.iterrows() :
    temp = (str(row['Patient_ID']),str(row['PerformedDate']))
    diag = (row['Name_calc'],row['TestResult_calc'])
    if temp not in AllVisits.keys():
        AllVisits[temp] = []
    AllVisits[temp] = AllVisits[temp] + [diag]

PVDF = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in AllVisits.items() ]))
PVDF = PVDF.transpose()
PVDF = PVDF.dropna(axis=0,how='all') # Remove NA


In [134]:
PVDF

0   \
1000100000266  2006-02-07 00:00:00          (Plasma Creatinine Clearance (eGFR), 1.0)   
               2006-02-13 00:00:00                                (BMI (kg/m^2), 2.0)   
               2006-05-11 00:00:00  (Alanine Aminotransferase in Serum or Plasma (...   
1000100001148  2010-01-05 00:00:00                   (Leukocytes in Blood (WBC), 2.0)   
1000100002076  2010-07-30 00:00:00  (Alanine Aminotransferase in Serum or Plasma (...   
...                                                                               ...   
12120120066987 2017-09-25 00:00:00                                (BMI (kg/m^2), 1.0)   
               2017-10-02 00:00:00                             (Hemoglobin (Hb), 1.0)   
               2018-02-02 00:00:00                      (Hemoglobin A1c (HbA1c), 2.0)   
               2018-03-20 00:00:00                             (Hemoglobin (Hb), 1.0)   
12120120078318 2019-10-17 00:00:00                      (Hemoglobin A1c (HbA1c), 2.0)   

                                                                    1   \
1000100000266  2006-02-07 00:00:00      (Total Cholesterol (TCh), 1.0)   
               2006-02-13 00:00:00                                 NaN   
               2006-05-11 00:00:00       (Hemoglobin A1c (HbA1c), 1.0)   
1000100001148  2010-01-05 00:00:00  (Erythrocytes in Blood (RBC), 3.0)   
1000100002076  2010-07-30 00:00:00              (Hemoglobin (Hb), 1.0)   
...                                                                ...   
12120120066987 2017-09-25 00:00:00                                 NaN   
               2017-10-02 00:00:00                                 NaN   
               2018-02-02 00:00:00                                 NaN   
               2018-03-20 00:00:00                                 NaN   
12120120078318 2019-10-17 00:00:00                                 NaN   

                                                                           2   \
1000100000266  2006-02-07 00:00:00         (Fasting Blood Glucose (FBG), 1.0)   
               2006-02-13 00:00:00                                        NaN   
               2006-05-11 00:00:00  (Plasma Creatinine Clearance (eGFR), 1.0)   
1000100001148  2010-01-05 00:00:00                     (Hemoglobin (Hb), 3.0)   
1000100002076  2010-07-30 00:00:00            (Platelets in Blood (PLT), 3.0)   
...                                                                       ...   
12120120066987 2017-09-25 00:00:00                                        NaN   
               2017-10-02 00:00:00                                        NaN   
               2018-02-02 00:00:00                                        NaN   
               2018-03-20 00:00:00                                        NaN   
12120120078318 2019-10-17 00:00:00                                        NaN   

                                                                                   3   \
1000100000266  2006-02-07 00:00:00  (Alanine Aminotransferase in Serum or Plasma (...   
               2006-02-13 00:00:00                                                NaN   
               2006-05-11 00:00:00                                                NaN   
1000100001148  2010-01-05 00:00:00                    (Platelets in Blood (PLT), 2.0)   
1000100002076  2010-07-30 00:00:00                   (Leukocytes in Blood (WBC), 3.0)   
...                                                                               ...   
12120120066987 2017-09-25 00:00:00                                                NaN   
               2017-10-02 00:00:00                                                NaN   
               2018-02-02 00:00:00                                                NaN   
               2018-03-20 00:00:00                                                NaN   
12120120078318 2019-10-17 00:00:00                                                NaN   

                                                                                   

In [135]:
eTests = []
for t in tests :
    for i in range(1,4) :
        eTests.append(t+str(i))

In [136]:
RowFeatures = pd.DataFrame(data = 0,
                       index = PVDF.index,
                       columns=eTests)
RowFeatures

Alanine Aminotransferase in Serum or Plasma (ALT)1  \
1000100000266  2006-02-07 00:00:00                                                  0    
               2006-02-13 00:00:00                                                  0    
               2006-05-11 00:00:00                                                  0    
1000100001148  2010-01-05 00:00:00                                                  0    
1000100002076  2010-07-30 00:00:00                                                  0    
...                                                                               ...    
12120120066987 2017-09-25 00:00:00                                                  0    
               2017-10-02 00:00:00                                                  0    
               2018-02-02 00:00:00                                                  0    
               2018-03-20 00:00:00                                                  0    
12120120078318 2019-10-17 00:00:00                                                  0    

                                    Alanine Aminotransferase in Serum or Plasma (ALT)2  \
1000100000266  2006-02-07 00:00:00                                                  0    
               2006-02-13 00:00:00                                                  0    
               2006-05-11 00:00:00                                                  0    
1000100001148  2010-01-05 00:00:00                                                  0    
1000100002076  2010-07-30 00:00:00                                                  0    
...                                                                               ...    
12120120066987 2017-09-25 00:00:00                                                  0    
               2017-10-02 00:00:00                                                  0    
               2018-02-02 00:00:00                                                  0    
               2018-03-20 00:00:00                                                  0    
12120120078318 2019-10-17 00:00:00                                                  0    

                                    Alanine Aminotransferase in Serum or Plasma (ALT)3  \
1000100000266  2006-02-07 00:00:00                                                  0    
               2006-02-13 00:00:00                                                  0    
               2006-05-11 00:00:00                                                  0    
1000100001148  2010-01-05 00:00:00                                                  0    
1000100002076  2010-07-30 00:00:00                                                  0    
...                                                                               ...    
12120120066987 2017-09-25 00:00:00                                                  0    
               2017-10-02 00:00:00                                                  0    
               2018-02-02 00:00:00                                                  0    
               2018-03-20 00:00:00                                                  0    
12120120078318 2019-10-17 00:00:00                                                  0    

                                    Aspartate Aminotransferase in Serum or Plasma (AST)1  \
1000100000266  2006-02-07 00:00:00                                                  0      
               2006-02-13 00:00:00                                                  0      
               2006-05-11 00:00:00                                                  0      
1000100001148  2010-01-05 00:00:00                                                  0      
1000100002076  2010-07-30 00:00:00                                                  0      
...                                                                               ...      
12120120066987 2017-09-25 00:00:00                                                  0      
               2017-10-02 00:00:00                       

In [137]:
for index, row in RowFeatures.iterrows() :
    temp = PVDF.loc[index].dropna()
    for t in temp :
        test = t[0] + str(int(t[1]))
        row[test] = 1

In [138]:
RowFeatures

Alanine Aminotransferase in Serum or Plasma (ALT)1  \
1000100000266  2006-02-07 00:00:00                                                  0    
               2006-02-13 00:00:00                                                  0    
               2006-05-11 00:00:00                                                  0    
1000100001148  2010-01-05 00:00:00                                                  0    
1000100002076  2010-07-30 00:00:00                                                  1    
...                                                                               ...    
12120120066987 2017-09-25 00:00:00                                                  0    
               2017-10-02 00:00:00                                                  0    
               2018-02-02 00:00:00                                                  0    
               2018-03-20 00:00:00                                                  0    
12120120078318 2019-10-17 00:00:00                                                  0    

                                    Alanine Aminotransferase in Serum or Plasma (ALT)2  \
1000100000266  2006-02-07 00:00:00                                                  0    
               2006-02-13 00:00:00                                                  0    
               2006-05-11 00:00:00                                                  0    
1000100001148  2010-01-05 00:00:00                                                  0    
1000100002076  2010-07-30 00:00:00                                                  0    
...                                                                               ...    
12120120066987 2017-09-25 00:00:00                                                  0    
               2017-10-02 00:00:00                                                  0    
               2018-02-02 00:00:00                                                  0    
               2018-03-20 00:00:00                                                  0    
12120120078318 2019-10-17 00:00:00                                                  0    

                                    Alanine Aminotransferase in Serum or Plasma (ALT)3  \
1000100000266  2006-02-07 00:00:00                                                  1    
               2006-02-13 00:00:00                                                  0    
               2006-05-11 00:00:00                                                  1    
1000100001148  2010-01-05 00:00:00                                                  0    
1000100002076  2010-07-30 00:00:00                                                  0    
...                                                                               ...    
12120120066987 2017-09-25 00:00:00                                                  0    
               2017-10-02 00:00:00                                                  0    
               2018-02-02 00:00:00                                                  0    
               2018-03-20 00:00:00                                                  0    
12120120078318 2019-10-17 00:00:00                                                  0    

                                    Aspartate Aminotransferase in Serum or Plasma (AST)1  \
1000100000266  2006-02-07 00:00:00                                                  0      
               2006-02-13 00:00:00                                                  0      
               2006-05-11 00:00:00                                                  0      
1000100001148  2010-01-05 00:00:00                                                  0      
1000100002076  2010-07-30 00:00:00                                                  0      
...                                                                               ...      
12120120066987 2017-09-25 00:00:00                                                  0      
               2017-10-02 00:00:00                       

In [139]:
RowFeatures.iloc[0]

Alanine Aminotransferase in Serum or Plasma (ALT)1      0
Alanine Aminotransferase in Serum or Plasma (ALT)2      0
Alanine Aminotransferase in Serum or Plasma (ALT)3      1
Aspartate Aminotransferase in Serum or Plasma (AST)1    0
Aspartate Aminotransferase in Serum or Plasma (AST)2    0
Aspartate Aminotransferase in Serum or Plasma (AST)3    0
Urea (mM)1                                              0
Urea (mM)2                                              0
Urea (mM)3                                              0
Plasma Creatinine Clearance (eGFR)1                     1
Plasma Creatinine Clearance (eGFR)2                     0
Plasma Creatinine Clearance (eGFR)3                     0
Potassium (mM)1                                         0
Potassium (mM)2                                         0
Potassium (mM)3                                         0
Sodium (mM)1                                            0
Sodium (mM)2                                            0
Sodium (mM)3  

In [140]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 18)
visits_emb = svd.fit_transform(RowFeatures)
reducedDF = pd.DataFrame(data=visits_emb, index=RowFeatures.index)

In [141]:
reducedDF.head()

0         1         2         3   \
1000100000266 2006-02-07 00:00:00  1.880463  0.768853  1.080596 -0.039715   
              2006-02-13 00:00:00  0.006754 -0.000856 -0.007786  0.000383   
              2006-05-11 00:00:00  0.526878 -0.133183  0.580864  0.179873   
1000100001148 2010-01-05 00:00:00  1.131989  1.070029  0.179398 -0.101280   
1000100002076 2010-07-30 00:00:00  1.067177 -0.731632  0.545187 -0.663878   

                                         4         5         6         7   \
1000100000266 2006-02-07 00:00:00 -0.256141  0.424808 -0.296227  0.858374   
              2006-02-13 00:00:00 -0.000462  0.002111  0.015879 -0.001309   
              2006-05-11 00:00:00 -0.335081 -0.150549  0.223647  0.155320   
1000100001148 2010-01-05 00:00:00  0.493376 -0.454107 -0.855253  0.467363   
1000100002076 2010-07-30 00:00:00 -0.091612 -0.797872  0.163430 -0.532386   

                                         8         9         10        11  \
1000100000266 2006-02-07 00:00:00 -0.647577 -0.350432  0.866445 -0.457067   
              2006-02-13 00:00:00 -0.001416 -0.004576  0.007697  0.000833   
              2006-05-11 00:00:00 -0.619480 -0.207016  0.047830 -0.218704   
1000100001148 2010-01-05 00:00:00 -0.162066  0.517175  0.019711  0.035751   
1000100002076 2010-07-30 00:00:00 -0.408658 -0.727252  0.737262 -0.420548   

                                         12        13        14        15  \
1000100000266 2006-02-07 00:00:00 -0.066770 -0.436668 -0.220422 -0.663002   
              2006-02-13 00:00:00  0.004533  0.008825 -0.008433  0.004166   
              2006-05-11 00:00:00 -0.363506 -0.599640 -0.114148 -0.105771   
1000100001148 2010-01-05 00:00:00 -0.235329  0.541359 -0.180945 -0.017347   
1000100002076 2010-07-30 00:00:00  0.181639 -0.247566  0.468600 -0.651515   

                                         16        17  
1000100000266 2006-02-07 00:00:00 -0.548356  0.254453  
              2006-02-13 00:00:00 -0.004116 -0.055236  
              2006-05-11 00:00:00 -0.665966  0.345086  
1000100001148 2010-01-05 00:00:00  0.358775 -0.162429  
1000100002076 2010-07-30 00:00:00  0.473148  0.192889

In [142]:
IDs = neg_samp

In [143]:
VisitHistorySVD = {}
for index, row in reducedDF.iterrows() :
    temp = int(index[0])
    featureVector = list(row)
    if temp not in VisitHistorySVD.keys():
        if temp in IDs :
            VisitHistorySVD[temp] = [0]
        else :
            VisitHistorySVD[temp] = [1]
    VisitHistorySVD[temp] = VisitHistorySVD[temp] + featureVector




In [144]:
VisitHistorySVD[1000100000266]

[1,
 1.880463373624588,
 0.7688527495824375,
 1.080596444217067,
 -0.039714740218250166,
 -0.2561406834875751,
 0.4248080588242752,
 -0.29622743892843173,
 0.8583739465125615,
 -0.6475774954730409,
 -0.35043245122945715,
 0.8664447927814402,
 -0.4570674250563151,
 -0.06676963836182029,
 -0.43666834802068083,
 -0.22042233269046752,
 -0.6630023528630097,
 -0.548355902922418,
 0.2544534432170209,
 0.006754274041419848,
 -0.0008555525543150817,
 -0.007786227429720518,
 0.00038328875250731776,
 -0.00046236943374533047,
 0.0021109313083772173,
 0.01587877967567929,
 -0.0013093397131585486,
 -0.0014161673747786945,
 -0.004576227528308205,
 0.007697460209326742,
 0.000833238671112466,
 0.004532947312851606,
 0.008824792639531382,
 -0.008432964847801134,
 0.004165850887038646,
 -0.0041162912432394895,
 -0.05523565798430808,
 0.5268780753785003,
 -0.1331831634282037,
 0.5808636076225706,
 0.17987254090787552,
 -0.3350814038840848,
 -0.15054853322755016,
 0.22364743691114516,
 0.1553196468384174,

In [145]:
m = [0]
for k,v in VisitHistorySVD.items() :
    temp = len(v)
    if temp > m[-1] :
        m.append(k)
        m.append(temp)
        
print(m)

[0, 1000100000266, 55, 1000100002076, 73, 1000100003603, 145, 1000100003708, 163, 1000100004129, 559, 1000100034421, 865, 5050080001678, 883]


In [146]:
# Fill in to 900
for k,v in VisitHistorySVD.items() :
    fillLen = 901 - len(v)
    VisitHistorySVD[k] = v + [0]*fillLen

In [147]:
# MODELING
# Convert data to x y
x, y = [], []
for k,v in VisitHistorySVD.items() :
    y.append(v[0])
    x.append(v[1:])

In [148]:
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D, GRU, Bidirectional, Input, TimeDistributed, Reshape

In [149]:
# Split data
x_train, x_test, y_train, y_test = \
    train_test_split(x, y, test_size = 0.1)
print(len(x_train), 'training')
print(len(x_test), 'testing')

2777 training
309 testing


In [150]:
# Bidirectional GRU

# Random search drop out rate and number of units/neurons
x_train = np.array(x_train)
x_test = np.array(x_test)
input_shape = (x_train.shape[1], 1) #900,1

units = 100


model = Sequential()
model.add(Reshape((50, 18), input_shape=(900,)))
# 1 sees the past 
# 2 sees the past and the future


dropout = 0.2 # for regularization
print('Number of hidden units: ', units, 'Dropout: ', dropout)
model.add(Bidirectional(LSTM(units, input_shape=input_shape))) # 64, $128$, 256, 512
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
print('Training...')
history = model.fit(
    np.array(x_train),
    np.array(y_train),
    batch_size = 128, 
    epochs = 15,
    validation_split = 0.1
)




Number of hidden units:  100 Dropout:  0.2
Training...
Epoch 1/15
20/20 [==============================] - 10s 193ms/step - loss: 0.6921 - accuracy: 0.5341 - val_loss: 0.6832 - val_accuracy: 0.5612
Epoch 2/15
20/20 [==============================] - 2s 84ms/step - loss: 0.6848 - accuracy: 0.5692 - val_loss: 0.6764 - val_accuracy: 0.5755
Epoch 3/15
20/20 [==============================] - 1s 71ms/step - loss: 0.6780 - accuracy: 0.5653 - val_loss: 0.6659 - val_accuracy: 0.5971
Epoch 4/15
20/20 [==============================] - 1s 73ms/step - loss: 0.6732 - accuracy: 0.5852 - val_loss: 0.6708 - val_accuracy: 0.5971
Epoch 5/15
20/20 [==============================] - 1s 71ms/step - loss: 0.6667 - accuracy: 0.6015 - val_loss: 0.6640 - val_accuracy: 0.6043
Epoch 6/15
20/20 [==============================] - 1s 71ms/step - loss: 0.6635 - accuracy: 0.6022 - val_loss: 0.6754 - val_accuracy: 0.5576
Epoch 7/15
20/20 [==============================] - 1s 74ms/step - loss: 0.6661 - accuracy: 0.587

In [151]:
print(model.evaluate(x_train, np.array(y_train)))
preds = model.predict(x_test)

87/87 [==============================] - 1s 10ms/step - loss: 0.6290 - accuracy: 0.6269
[0.6290353536605835, 0.6269355416297913]


In [152]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds)

0.6164488202342557